In [1]:
# Importaciones
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
years = [1980,1981,1982,1983,1984,1985,1986,1987,1988,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

In [3]:
def get_year(year):
  # Crear el dataframe
  df = pd.DataFrame(columns=['Fecha','Ubicacion','Local','Resultado','Visitante'])
  web = f'https://es.wikipedia.org/wiki/Copa_Libertadores_{year}'
  response = requests.get(web)
  content = response.text
  soup = BeautifulSoup(content, 'lxml')

  tables = soup.find_all('table', {'class':'wikitable collapsible collapsed'})

  for table in tables:
    games = table.find_all('tr', align='center')
    for i in range(len(games)):
      resume_game = games[i].get_text().splitlines()
      resume_game = [elemento for elemento in resume_game if elemento != '']
      if len(resume_game) == 5:
        df.loc[len(df)] = resume_game

  final = soup.find_all('table', {'class':'collapsible autocollapse vevent plainlist'})

  for instance in final:
    a = [e for e in instance.get_text().splitlines() if e != '']
    # Solo seleccionar la data necesaria
    a = a[:5]
    # Cambiar de lugar la ubicacion del partido
    aux = a.pop(4)
    a.insert(1,aux)
    # Eliminar todo lo de antes de la coma
    a[1] = a[1].split(',')[1].strip()
    # Eliminar espacios vacios
    for i in range(5):
      a[i] = a[i].strip()
    # Eliminar resultado global
    a[3] = a[3].split('(')[0].strip()
    df.loc[len(df)] = a
  # Agregar esa ultima columna llamada year
  df['year'] = year
  return df

# Ejecutar la funcion
fifa = [get_year(year) for year in years]
df_fifa = pd.concat(fifa, ignore_index=True)

# Fixture de la libertadores actual
fixture = get_year(2023)

In [3]:
df_fifa.to_csv('historial.csv', index=False)
fixture.to_csv('fixture.csv', index=False)